# Steady State - Aggregate

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Find steady state](#toc2_)    
- 3. [Speed and error tests](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from BabyMAKROModel import BabyMAKROModelClass

C:\Users\nbk863\AppData\Local\Temp\ipykernel_9580\3413739869.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [2]:
model = BabyMAKROModelClass(name='baseline') #create an instance of the model

## 2. <a id='toc2_'></a>[Find steady state](#toc0_)

In [3]:
model.find_ss(do_print=True)    #find steady state

Labor supply, search and matching:
ss.S/par.N_work = 0.15, ss.L/par.N_work = 0.96, ss.U/par.N_work = 0.04
ss.delta_L = 0.12, ss.m_s = 0.75, ss.m_v = 0.75, ss.v = 7.58
Capital agency FOC:
ss.r_K = 0.14
Labor agency FOC:
ss.r_ell = 1.01, (ss.L-ss.ell)/par.N_work*100 = 0.76
Production firm:
ss.P_Y_0 = 0.91, ss.Gamma = 0.57, ss.Y = 80.09, ss.K = 175.64
Capital accumulation:
ss.iota = 17.56, ss.I = 17.56
Government:
ss.B = 0.00, ss.G = 24.03, ss.tau = 0.58
Households:
ss.Aq/par.N = 0.04, ss.real_W = 1.00, ss.C = 26.28, ss.A = 55.00, ss.r_hh = 0.04
Market clearing:
ss.C_Y/ss.C = 0.70, ss.G_Y/ss.G = 0.90, ss.I_Y/ss.I = 0.65, ss.X_Y/ss.X = 0.60
ss.X/ss.Y = 0.60, ss.M/ss.Y = 0.44
Bargaining:
par.phi = 0.643


## 3. <a id='toc3_'></a>[Speed and error tests](#toc0_)

In [4]:
model.set_exo_ss()          #set steady state exogenous variables 
model.set_unknowns_ss()     #set steady state unknown variables

**Python evaluation:**

In [5]:
%time model.evaluate_blocks(py=True)

CPU times: total: 2.28 s
Wall time: 2.28 s


**Check errors:**

In [6]:
errors = model.get_errors(do_print=True)

A_R_ini_error       : abs. max = 1.48e-14
Aq_diff             : abs. max = 5.37e-14
FOC_capital_agency  : abs. max = 4.44e-16
FOC_K_ell           : abs. max = 8.88e-16
mkt_clearing        : abs. max = 1.42e-14
PC                  : abs. max = 0.00e+00
bargaining_cond     : abs. max = 0.00e+00


In [7]:
if np.allclose(errors,0.0):
    print('errors are zero')
else:
    raise ValueError('errors are NOT zero')

errors are zero


**First numba evaluation:**

In [8]:
%time model.evaluate_blocks()

CPU times: total: 34.2 s
Wall time: 34.7 s


**Time:**

In [9]:
%timeit model.evaluate_blocks()
%timeit model.get_errors()

10.9 ms ± 312 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
29.4 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%timeit model.evaluate_blocks(py=True)

200 ms ± 5.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
